Course: CS4487 project  
Group 16  
Student Name: Fung Sze Man  (SID: 56344553)  
Student Name: Chiu Chun Choi (SID: 56666173)

# Requirements.txt:  
torch  
torchvision  
numpy  
matplotlib==3.8.2  
tqdm==4.66.1  
efficientnet-pytorch==0.7.1  
scikit-learn==1.3.2  

# Testing the model  
please run the cells for testing after the following steps are done  

0. please make sure the following folder structure, create these under root dir if not exist:   
    - ./Datasets
    - ./results
1. install the requirements listed above
2. put the model under "./results", please make sure the model_save_name is changed to the path under "./results" where the model is stored for testing and adjust other related configurations if needed 
3. put the test dataset in ./Dataset   
4. change the "val" in following line in the 4th cell below to the subfolder name containing test set e.g if the subfolder is "test" then change to "test"
"test_dataset = datasets.ImageFolder(os.path.join(test_config["data_root"], "val"), transform=transform"

In [ ]:
import os
import efficientnet_pytorch
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
import numpy as np

Altering the output from EffecientNet for our train/test task

In [2]:
class CustomEfficientNet(nn.Module):
    def __init__(self, num_class):
        super(CustomEfficientNet, self).__init__()
        self.base_model = efficientnet_pytorch.EfficientNet.from_pretrained(
            'efficientnet-b0'
        )
        self.base_model._fc = nn.Linear(
            in_features=self.base_model._fc.in_features, 
            out_features=num_class, 
            bias=True
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.base_model(x)
        return x

In [17]:
test_config = {
    "model_name": "efficientnet_b0",  # same as training configuration
    "batch_size": 8,  # can be different from training batch size
    "data_root": "../Dataset",  # root directory of the test dataset
    "output_dir": "./results",  # directory where the model is saved
    "gpus": [0],               # Default is GPU 0, change to [0, 1] for both GPUs
    "model_save_name": "efficientnet_b0-20231208094757/efficientnet_b0-20231208094757.pth"  # name of the saved model file
}

load model function

In [18]:
def load_model(model_path, model, device):
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model

load the model with above configuration, test set, and prepare test set loader  

In [19]:
if test_config["model_name"].lower()[:3] == "vit":
    transform = transforms.Compose([
        transforms.Resize((224)),
        transforms.ToTensor(),
        transforms.Normalize([0.0291, 0.0269, 0.0253], [0.1319, 0.1239, 0.1194]),
        # to save computation cost use pre calculated mean and std here, for other datasets use getmeanstd()
    ])
else:
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.0291, 0.0269, 0.0253], [0.1319, 0.1239, 0.1194]),
        # to save computation cost use pre calculated mean and std here, for other datasets use getmeanstd()
    ])

test_dataset = datasets.ImageFolder(os.path.join(test_config["data_root"], "val"), transform=transform)
test_loader = DataLoader(test_dataset, batch_size=test_config["batch_size"], shuffle=False)

num_classes = len(test_dataset.classes)
out_features =  1 if num_classes < 3 else num_classes
print(out_features)

if test_config["model_name"].lower() == "vit_l_32":
    model = models.vit_l_32(pretrained=False)
elif test_config["model_name"].lower() == "swin_v2_b":
    model = models.swin_v2_b(pretrained=False)
elif test_config["model_name"].lower() == "efficientnet_b0":
    model = CustomEfficientNet(out_features)
else:
    raise ValueError("Unsupported model")

# GPU setup
device = torch.device(f"cuda:{test_config['gpus'][0]}" if torch.cuda.is_available() else "cpu")
model.to(device)

if len(config["gpus"]) > 1:
    model = nn.DataParallel(model, device_ids=config["gpus"])

# Load the trained model
model_path = os.path.join(test_config["output_dir"], test_config["model_save_name"])
model = load_model( model_path, model, device)

1
Loaded pretrained weights for efficientnet-b0


### The following cell is our test inference code

In [ ]:
def test(model, test_loader):
    model.eval()
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images)
            
            # different handle for multi classes
            if out_features > 1: 
                labels = labels.type(torch.LongTensor).to(device)
            else:
                labels = labels.unsqueeze(1)

            # different handle for multi classes
            if out_features > 1:
                predictions = torch.max(outputs,1).indices
            else:
                predictions = torch.sigmoid(outputs) > 0.5
                predictions = predictions.type(torch.LongTensor).to(device)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)

    accuracy = accuracy_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions, average='weighted')
    precision = precision_score(all_labels, all_predictions, average='weighted')
    auc = roc_auc_score(all_labels, all_predictions)

    print(f'Test Results\n Accuracy: {accuracy:.4f},\n Recall: {recall:.4f},\n Precision: {precision:.4f},\n AUC: {auc:.4f}')
    return accuracy, recall, precision, auc

# run testing
accuracy, recall, precision, auc = test(model, test_loader)